# QPT MixSIAR
Create MixSIAR input files

**Manuscript title**: Bayesian modeling of plant wax sources to an Eastern Canadian Arctic lake sediment record reveals stable plant wax vegetation sources following postglacial shrub colonization

**Manuscript authors**: Kurt R. Lindberg, Elizabeth K. Thomas, Martha K. Raynolds, Helga Bultmann

**DOI**": pending

In [1]:
## Import necessary Python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Function that generates MixSIAR source input files

def make_source(sources, source_names, data_type, start_chain, end_chain, scale):

  conc_data_type = data_type + "conc"
  iso_data_type = data_type + "d13c"
  chain_lengths = list(range(start_chain, end_chain+1))
  source_n = []
  tracers = {}

  names = pd.Series(data=source_names, name='source_names')

  for i in range(0, len(sources['Endmember'])):
    source_n.append(len(sources['Endmember'][i].growth_form))

  n_samples = pd.DataFrame(data=np.array(source_n), columns=['n'])

  match data_type:
    case "f":
        chain_lengths = [num for num in chain_lengths if num % 2 == 0]
    case "a":
        chain_lengths = [num for num in chain_lengths if num % 2 == 1]

  for n in chain_lengths:
    tracers.update(
        {
          f"Meanc{n}": [],
          f"SDc{n}": [],
          f"Concc{n}": []
        }
      )

    for end in range(0, len(sources['Endmember'])):
      df = sources['Endmember'][end]
      wax_conc_all = df.filter(regex=conc_data_type)
      wax_iso_all = df.filter(regex=iso_data_type)

      chain_conc = pd.DataFrame(data=np.array(wax_conc_all.filter(items=["c"+str(n)+"_"+conc_data_type])), columns=[str(n)])
      chain_iso = pd.DataFrame(data=np.array(wax_iso_all.filter(items=["c"+str(n)+"_"+iso_data_type])), columns=[str(n)])

      tracers[f"Meanc{n}"].append(np.nanmean(np.array(chain_iso)))
      tracers[f"SDc{n}"].append(np.nanstd(np.array(chain_iso)))
      tracers[f"Concc{n}"].append((np.nanmean(np.array(chain_conc)))*scale[end])

  mixsiar_tracers = pd.DataFrame(data=tracers)
  mixsiar_source = pd.concat([names, mixsiar_tracers, n_samples], axis=1)

  return mixsiar_source


# Function that generates MixSIAR discrimination factor (discr) input files

def make_discr(source_names, data_type, start_chain, end_chain):

  tracers = {}
  names = pd.Series(data=source_names, name='source_names')
  chain_lengths = list(range(start_chain, end_chain+1))

  match data_type:
    case "f":
        chain_lengths = [num for num in chain_lengths if num % 2 == 0]
    case "a":
        chain_lengths = [num for num in chain_lengths if num % 2 == 1]

  for n in chain_lengths:
    tracers.update(
      {
        f"Meanc{n}": np.zeros(len(source_names)),
        f"SDc{n}": np.zeros(len(source_names))
      }
    )

  mixsiar_tracers = pd.DataFrame(data=tracers)
  mixsiar_discr = pd.concat([names, mixsiar_tracers], axis=1)

  return mixsiar_discr

In [3]:
## Import raw plant wax data for endmember priors

# Lake QPT data only - Hollister et al. (2022)
# DOI: 
wax_data = pd.read_excel(
    'qpt_plantwax_source.xlsx',
    sheet_name='plantwax'
)

# Remove samples where concentration is not reported in ug/g plant
wax_data = wax_data[wax_data['conc_ugg_plant'] == 1]

In [4]:
## Define endmember groupings

# All terrestrial plants
terr = wax_data[wax_data['habitat'] == 'terrestrial'].reset_index(drop=True)

# Terrestrial vascular plants
tvas = terr[terr['growth_form'].str.contains("tree|shrub|forb|fern|graminoid")].reset_index(drop=True)

# Terrestrial shrubs
tshr = tvas[tvas['growth_form'] == 'shrub'].reset_index(drop=True)

# Terrestrial non-shrubs
tnshr = tvas[tvas['growth_form'] != 'shrub'].reset_index(drop=True)

# Terrestrial non-vascular plants
tnvas = terr[terr['growth_form'].str.contains("moss|liverwort|lichen")].reset_index(drop=True)

# All aquatic plants
aqua = wax_data[wax_data['habitat'] != 'terrestrial'].reset_index(drop=True)

# Aquatic non-vascular plants
anvas = aqua[aqua['growth_form'].str.contains("moss|algae")].reset_index(drop=True)

# Aquatic vascular plants
avas = aqua[aqua['growth_form'].str.contains("forb|graminoid")].reset_index(drop=True)

In [5]:
## Generate QPT-only MixSIAR source files

source_qpt_4end_c22c28 = make_source(
  sources={
    'Endmember': [
      aqua,
      tnvas,
      tnshr,
      tshr
    ]
  },
  source_names=['aqua','nonvas','nonshrub','shrub'],
  data_type="f",
  start_chain=22,
  end_chain=28,
  scale=[1,1,1,1]
).to_csv('source/source_qpt_4end_c22c28.csv', index=False)

source_qpt_3end_c22c28 = make_source(
  sources={
    'Endmember': [
    aqua,
    tnvas,
    tvas
    ]
  },
  source_names=['aqua','nonvas','tvas'],
  data_type="f",
  start_chain=22,
  end_chain=28,
  scale=[1,1,1]
).to_csv('source/source_qpt_3end_c22c28.csv', index=False)

source_qpt_3end_c22c24 = make_source(
  sources={
    'Endmember': [
    aqua,
    tnvas,
    tvas
    ]
  },
  source_names=['aqua','nonvas','tvas'],
  data_type="f",
  start_chain=22,
  end_chain=24,
  scale=[1,1,1]
).to_csv('source/source_qpt_3end_c22c24.csv', index=False)

source_qpt_3end_c26c28 = make_source(
  sources={
    'Endmember': [
    aqua,
    tnvas,
    tvas
    ]
  },
  source_names=['aqua','nonvas','tvas'],
  data_type="f",
  start_chain=26,
  end_chain=28,
  scale=[1,1,1]
).to_csv('source/source_qpt_3end_c26c28.csv', index=False)


source_qpt_2end_c22c28 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=22,
  end_chain=28,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c22c28.csv', index=False)

source_qpt_2end_c22c24 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=22,
  end_chain=24,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c22c24.csv', index=False)

source_qpt_2end_c26c28 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=26,
  end_chain=28,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c26c28.csv', index=False)

source_qpt_2end_c22 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=22,
  end_chain=22,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c22.csv', index=False)

source_qpt_2end_c24 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=24,
  end_chain=24,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c24.csv', index=False)

source_qpt_2end_c26 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=26,
  end_chain=26,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c26.csv', index=False)

source_qpt_2end_c28 = make_source(
  sources={
    'Endmember': [
      aqua,
      terr
    ]
  },
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=28,
  end_chain=28,
  scale=[1,1]
).to_csv('source/source_qpt_2end_c28.csv', index=False)

In [ ]:
## Generate ECA MixSIAR source files
# To be released

In [ ]:
## Generate discr files
# Independent of data selection, endmember types just need to match

discr_4end_c22c28 = make_discr(
  source_names=['aqua','nonvas','nonshrub','shrub'],
  data_type="f",
  start_chain=22,
  end_chain=28
).to_csv('discr/discr_4end_c22c28.csv', index=False)


discr_3end_c22c28 = make_discr(
  source_names=['aqua','nonvas','tvas'],
  data_type="f",
  start_chain=22,
  end_chain=28
).to_csv('discr/discr_3end_c22c28.csv', index=False)

discr_3end_c22c24 = make_discr(
  source_names=['aqua','nonvas','tvas'],
  data_type="f",
  start_chain=22,
  end_chain=24
).to_csv('discr/discr_3end_c22c24.csv', index=False)

discr_3end_c26c28 = make_discr(
  source_names=['aqua','nonvas','tvas'],
  data_type="f",
  start_chain=26,
  end_chain=28
).to_csv('discr/discr_3end_c26c28.csv', index=False)


discr_2end_c22c28 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=22,
  end_chain=28
).to_csv('discr/discr_2end_c22c28.csv', index=False)

discr_2end_c22c24 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=22,
  end_chain=24
).to_csv('discr/discr_2end_c22c24.csv', index=False)

discr_2end_c26c28 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=26,
  end_chain=28
).to_csv('discr/discr_2end_c26c28.csv', index=False)


discr_2end_c22 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=22,
  end_chain=22
).to_csv('discr/discr_2end_c22.csv', index=False)

discr_2end_c24 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=24,
  end_chain=24
).to_csv('discr/discr_2end_c24.csv', index=False)

discr_2end_c26 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=26,
  end_chain=26
).to_csv('discr/discr_2end_c26.csv', index=False)

discr_2end_c28 = make_discr(
  source_names=['aqua','terr'],
  data_type="f",
  start_chain=28,
  end_chain=28
).to_csv('discr/discr_2end_c28.csv', index=False)